Setup

In [2]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [3]:
SEASONS = list(range(2016,2024))
WEBSITE = f"https://www.basketball-reference.com"

In [4]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

Extracting HTML

In [5]:
async def get_html(url, selector, start_time, retries = 3, sleep = 5):
    html = None
    for i in range(0, retries):
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            cur_time = int(time.time()) - start_time
            cur_hrs = f"{int(cur_time / 3600)} hrs"
            cur_mins = f"{int(cur_time % 3600 / 60)} mins"
            cur_secs = f"{int(cur_time % 60)} secs"
            print(f"Timeout error on {url} at {cur_hrs} {cur_mins} {cur_secs}")
            continue
        else:
            break
    
    cur_time = int(time.time()) - start_time
    cur_hrs = f"{int(cur_time / 3600)} hrs"
    cur_mins = f"{int(cur_time % 3600 / 60)} mins"
    cur_secs = f"{int(cur_time % 60)} secs"

    print(f"Take a break at {cur_hrs} {cur_mins} {cur_secs}")
    print()
    time.sleep(sleep)
    return html


In [6]:
async def scrape_season(season, start_time):
    url = f"{WEBSITE}/leagues/NBA_{season}_games.html"

    html = await get_html(url, "#content .filter", start_time)

    if html is None:
        return
    
    soup = BeautifulSoup(html)
    
    links = soup.findAll("a")
    href = [l["href"] for l in links]
    standings_pages = [f"{WEBSITE}{l}" for l in href]
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule", start_time)
        with open(save_path, "w+") as f:
            f.write(html)

In [7]:
start_time = int(time.time())
for season in SEASONS:
    await scrape_season(season, start_time)
print(f"Done")

2015-16 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 0 mins 3 secs

2016-17 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 0 mins 10 secs

2017-18 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 0 mins 34 secs

2018-19 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 0 mins 41 secs

Timeout error on https://www.basketball-reference.com/leagues/NBA_2020_games.html at 0 hrs 1 mins 17 secs
2019-20 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 1 mins 31 secs

Timeout error on https://www.basketball-reference.com/leagues/NBA_2021_games.html at 0 hrs 2 mins 7 secs
2020-21 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 2 mins 17 secs

2021-22 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 2 mins 23 secs

2022-23 NBA Schedule | Basketball-Reference.com
Take a break at 0 hrs 2 mins 30 secs

Done


Get Box Scores

In [8]:
standings_files = os.listdir(STANDINGS_DIR)
standings_files = [s for s in standings_files if ".html" in s]
standings_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2017_games-february.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2023_games-december.html',
 'NBA_2017_games-april.html',
 'NBA_2022_games-november.html',
 'NBA_2016_games-january.ht

In [9]:
async def scrape_game(standings_file, start_time):
    with open(standings_file, "r") as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores_links = [f"{WEBSITE}{l}" for l in box_scores]

    for url in box_scores_links:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content", start_time)
        if not html:
            continue
        
        with open(save_path, "w+") as f:
            f.write(html)


In [10]:
start_time = int(time.time())
for f in standings_files:
    file_path = os.path.join(STANDINGS_DIR, f)
    await scrape_game(file_path, start_time)
print(f"Done")

Done
